# HORTON3 Notebook

In [1]:
import numpy as np

## IOData

For more information check [IOData GitHub repository](https://github.com/theochem/iodata)

In [2]:
from iodata import load_one

In [3]:
# load FCHK file
mol = load_one("data/h2o_sto3g.fchk")

print("Number of atoms    = ", mol.natom)
print("Atomic numbers     = ", mol.atnums)
print("Atomic coordinates = ")
print(mol.atcoords)

Number of atoms    =  3
Atomic numbers     =  [8 1 1]
Atomic coordinates = 
[[-4.44734101  3.39697999  0.        ]
 [-2.58401495  3.55136194  0.        ]
 [-4.92380519  5.2049622   0.        ]]


## Grid

For more information check [Grid GitHub repository](https://github.com/theochem/grid)

In [4]:
from grid.becke import BeckeWeights
from grid.molgrid import MolGrid
from grid.onedgrid import GaussChebyshev
from grid.rtransform import BeckeTF

In [5]:
# Make molecular grid (using grid package)
oned = GaussChebyshev(100)
rgrid = BeckeTF(1e-4, 1.5).transform_1d_grid(oned)
grid = MolGrid.from_size(mol.atnums, mol.atcoords, rgrid, 110, BeckeWeights())

print("Grid points shape  = ", grid.points.shape)
print("Grid weights shape = ", grid.weights.shape)

Grid points shape  =  (33000, 3)
Grid weights shape =  (33000,)


## GBasis

For more information check [GBasis GitHub repository](https://github.com/theochem/gbasis)

In [6]:
from gbasis.wrappers import from_iodata
from gbasis.evals.density import evaluate_density

In [7]:
# Compute molecular density (using gbasis package)
one_rdm = mol.one_rdms.get("post_scf", mol.one_rdms.get("scf"))
if one_rdm is None:
    if mol.mo is None:
        raise ValueError(
            "The input file lacks wavefunction data with which "
            "the density can be computed."
        )
    coeffs, occs = mol.mo.coeffs, mol.mo.occs
    one_rdm = np.dot(coeffs * occs, coeffs.T)
basis = from_iodata(mol)
density = evaluate_density(one_rdm, basis[0], grid.points, coord_type=basis[1])

print("Density shape = ", density.shape)

Density shape =  (33000,)


## Denspart

For more information check [Denspart GitHub repository](https://github.com/theochem/denspart)

In [8]:
from denspart.mbis import MBISProModel
from denspart.vh import optimize_reduce_pro_model
from denspart.properties import compute_radial_moments, compute_multipole_moments, safe_ratio

In [9]:
# MBIS partitioning (using denspart package)
pro_model_init = MBISProModel.from_geometry(mol.atnums, mol.atcoords)

pro_model, localgrids = optimize_reduce_pro_model(
    pro_model_init,
    grid,
    density,
    # args.gtol,
    # args.maxiter,
    # args.density_cutoff,
)
print("Promodel")
pro_model.pprint()

print("Computing additional properties")
results = pro_model.to_dict()

results.update(
    {
        "charges": pro_model.charges,
        "radial_moments": compute_radial_moments(
            pro_model, grid, density, localgrids
        ),
        "multipole_moments": compute_multipole_moments(
            pro_model, grid, density, localgrids
        ),
        # "gtol": args.gtol,
        # "maxiter": args.maxiter,
        # "density_cutoff": args.density_cutoff,
    }
)
# np.savez_compressed(args.out_npz, **results)

Building local grids
Integral of density: 10.000088051536862
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.5936396    0.5937277  -8.8052e-05   2.4770e-01    0.7150370
    2      6    0.3753745    0.2911400   8.4234e-02   1.2533e-01    0.0141320
    3     12    0.2686959    0.1705652   9.8131e-02   6.2372e-02    0.0141530
    4     12    0.2686959    0.1705652   9.8131e-02   6.2372e-02    0.0141530
    5     19    0.2305333    0.1914990   3.9034e-02   1.2143e-02    0.0139700
    6     19    0.2305333    0.1914990   3.9034e-02   1.2143e-02    0.0139700
    7     28    0.2244088    0.2004429   2.3966e-02   2.7924e-03    0.0153210
    8     28    0.2244088    0.2004429   2.3966e-02   2.7924e-03    0.0153210
    9     35    0.2242477    0.2167737   7.4740e-03   7.5061e-04    0.0145030
   10     35    0.2242477    0.2167737   7.4740e-03   7.5061e-04    

### Atomic Weights

In [10]:
# Compute atomic weights
pro = pro_model.compute_density(grid, localgrids)
# result = np.zeros((pro_model.natom, nmax + 1))
for iatom, atcoord in enumerate(pro_model.atcoords):
    print("Atom index = ", iatom)
    localgrid = grid.get_localgrid(atcoord, 8.0)
    pro_atom = pro_model.compute_proatom(iatom, localgrid.points)
    # ratio (copied from denspart properties module) is the ratio of molecular and promolecular density
    # ratio = safe_ratio(density[localgrid.indices], pro[localgrid.indices])
    # atomic weight
    atweight = safe_ratio(pro_atom, pro[localgrid.indices])
    # check: compute atomic population to make sure it matches expected values (from Denspart)
    print(" Computed Charge = ", mol.atnums[iatom] - localgrid.integrate(atweight * density[localgrid.indices]))
    print(" Expected Charge = ", pro_model.charges[iatom])
    print("")

Atom index =  0
 Computed Charge =  -0.5432644398016997
 Expected Charge =  -0.543264566552871

Atom index =  1
 Computed Charge =  0.2716629962211814
 Expected Charge =  0.2716629419119454

Atom index =  2
 Computed Charge =  0.27151342808814816
 Expected Charge =  0.2715133736087356

